We will assign each item found in the crate tree a unique index, i.e. a seperate row of a pandas dataframe. In each row of the df, the following information will be stored: 
    1. The letter stored in that crate
    2. The column that crate currently belongs to
    3. The position in the column that the crate belongs to (1 is lowest, n is highest)
    
Each instruction will be read sequentially. The statements are read in the form: move X from Y to Z:
    1. X will be the number of times the for loop is executed
    2. Y will be the column that crates are taken from
    3. Z will be the column that crates are reassigned to
    
We will require the following functions which can carry out the relevant operations:
    1. get_top_crate(column): find the row index and corresponding height in the table that corresponds to the highest crate in that column 
    2. move_crate(from_index, target_column, target_column_height): move the crate to the target column by reassigning its current column and its current height
    
We will follow the procedure:
    1. Read an instruction and parse the required information. 
    2. Find the row in the table that corresponds to the item that must be moved.
    3. Find the height of the target column. 
    4. Move the item to the target column.
    5. Repeat steps 2-5 as many times as required.
    6. Repeat steps 1-6 for every instruction. 

## Read in the input

In [1]:
with open("input.txt") as input_file:    
     data = input_file.read()
        
data

'[N]             [R]             [C]\n[T] [J]         [S] [J]         [N]\n[B] [Z]     [H] [M] [Z]         [D]\n[S] [P]     [G] [L] [H] [Z]     [T]\n[Q] [D]     [F] [D] [V] [L] [S] [M]\n[H] [F] [V] [J] [C] [W] [P] [W] [L]\n[G] [S] [H] [Z] [Z] [T] [F] [V] [H]\n[R] [H] [Z] [M] [T] [M] [T] [Q] [W]\n 1   2   3   4   5   6   7   8   9 \n\nmove 3 from 9 to 7\nmove 4 from 4 to 5\nmove 2 from 4 to 6\nmove 4 from 7 to 5\nmove 3 from 7 to 3\nmove 2 from 5 to 9\nmove 5 from 6 to 3\nmove 5 from 9 to 1\nmove 3 from 8 to 4\nmove 3 from 4 to 6\nmove 8 from 1 to 8\nmove 1 from 8 to 6\nmove 2 from 8 to 2\nmove 5 from 8 to 4\nmove 1 from 8 to 1\nmove 6 from 6 to 4\nmove 1 from 7 to 9\nmove 5 from 1 to 7\nmove 1 from 1 to 2\nmove 2 from 9 to 8\nmove 6 from 4 to 9\nmove 1 from 6 to 8\nmove 3 from 2 to 7\nmove 4 from 2 to 8\nmove 4 from 9 to 3\nmove 6 from 5 to 4\nmove 7 from 8 to 1\nmove 10 from 4 to 1\nmove 12 from 1 to 5\nmove 1 from 4 to 9\nmove 1 from 2 to 3\nmove 2 from 9 to 1\nmove 1 from 9 to 3\nmo

## Read and format input 

After trying to use regex and transforms, I found in the end it was easier to just type in the input into the pandas DataFrame format I needed. This isn't ideal and is hacky but it does work. We will enter the data as: content, column, height. 

In [2]:
import pandas as pd
input = zip(['N','T','B','S','Q','H','G','R' ,'J','Z','P','D','F','S','H', 'V','H','Z', 'H','G','F','J','Z','M', 'R','S','M','L','D','C','Z','T', 'J','Z','H','V','W','T','M', 'Z','L','P','F','T', 'S','W','V','Q' ,'C','N','D','T','M','L','H','W'],
            [1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,3,3,3,4,4,4,4,4,4,5,5,5,5,5,5,5,5,6,6,6,6,6,6,6,7,7,7,7,7,8,8,8,8,9,9,9,9,9,9,9,9],
            [8,7,6,5,4,3,2,1,7,6,5,4,3,2,1,3,2,1,6,5,4,3,2,1,8,7,6,5,4,3,2,1,7,6,5,4,3,2,1,5,4,3,2,1,4,3,2,1,8,7,6,5,4,3,2,1])

df = pd.DataFrame(input, columns=['Contents', 'Column', 'Height'])
df1 = df.copy()
df1

,Contents,Column,Height
0,N,1,8
1,T,1,7
2,B,1,6
3,S,1,5
4,Q,1,4
5,H,1,3
6,G,1,2
7,R,1,1
8,J,2,7
9,Z,2,6


Now read the set of instructions for the crate operations

In [3]:
import re
regex_instructions = r"\w{4}\s(\d+)\s\w{4}\s(\d+)\s\w{2}\s(\d+)"
instructions = re.findall(regex_instructions, data)
instructions

[('3', '9', '7'),
 ('4', '4', '5'),
 ('2', '4', '6'),
 ('4', '7', '5'),
 ('3', '7', '3'),
 ('2', '5', '9'),
 ('5', '6', '3'),
 ('5', '9', '1'),
 ('3', '8', '4'),
 ('3', '4', '6'),
 ('8', '1', '8'),
 ('1', '8', '6'),
 ('2', '8', '2'),
 ('5', '8', '4'),
 ('1', '8', '1'),
 ('6', '6', '4'),
 ('1', '7', '9'),
 ('5', '1', '7'),
 ('1', '1', '2'),
 ('2', '9', '8'),
 ('6', '4', '9'),
 ('1', '6', '8'),
 ('3', '2', '7'),
 ('4', '2', '8'),
 ('4', '9', '3'),
 ('6', '5', '4'),
 ('7', '8', '1'),
 ('10', '4', '1'),
 ('12', '1', '5'),
 ('1', '4', '9'),
 ('1', '2', '3'),
 ('2', '9', '1'),
 ('1', '9', '3'),
 ('1', '6', '7'),
 ('1', '9', '1'),
 ('3', '1', '3'),
 ('9', '5', '9'),
 ('2', '2', '7'),
 ('2', '7', '4'),
 ('3', '9', '4'),
 ('7', '5', '7'),
 ('5', '1', '3'),
 ('2', '4', '5'),
 ('1', '4', '6'),
 ('1', '6', '9'),
 ('4', '9', '2'),
 ('12', '7', '9'),
 ('2', '4', '9'),
 ('6', '5', '9'),
 ('3', '7', '6'),
 ('12', '9', '6'),
 ('5', '9', '1'),
 ('1', '7', '6'),
 ('14', '6', '1'),
 ('20', '3', '5'),
 ('5

## Defining functions

Now we define the functions we need. As we specified earlier, we need a function that returns the index, height and contents of the heighest crate in a given column. Again this is a little bit hacky but it works.

We also need a move crate function that can move a given crate (specified by its index in the Pandas dataframe) to another column. 

In [4]:
def get_top_crate(df, column):
    # Keep only the crates that are in the column of interest
    column_crates = df[df['Column'] == column]
    
    # If there are no crates in that column then return the height as zero
    if column_crates.empty:
        return None, 0, None
        
    # Find the heighest crate in that column
    heighest_crate = column_crates.loc[column_crates['Height'] == column_crates.max()['Height']] 
    
    # Return the row index, corresponding height and contents of the heighest crate in that column
    return heighest_crate.index[0], heighest_crate.iloc[0]['Height'], heighest_crate.iloc[0]['Contents']
    
def move_crate(df, source_index, target_column):
    # Find the current height of the target column
    _,current_height,_ = get_top_crate(df, target_column)
    
    # Reassign the crate's column and height values accordingly
    df.loc[df.index == source_index, 'Column'] = target_column
    df.loc[df.index == source_index, 'Height'] = current_height+1

## Executing instructions

We can now define a function which executes a single instruction using the functions we made earlier

In [5]:
def do_instruction(df, instruction):
    for _ in range(0, int(instruction[0])):
        # Find the crate that we need to move
        source_index,_,_ = get_top_crate(df, int(instruction[1]))

        # Move the crate to the target column
        move_crate(df, source_index, int(instruction[2]))

We need to carry out every instruction in our list and track the result of the movements. Again, using sets of for loops isn't ideal but its simple

In [6]:
for instruction in instructions:
    do_instruction(df1, instruction)

Finally, the answer is given by reading the value in the top crate of each column

In [7]:
answer1 = ""
for column in range(1, df1.max()['Column']+1):
    _,_,top_crate_content = get_top_crate(df1, column)
    answer1 += top_crate_content
    
answer1

'PTWLTDSJV'

## Part 2 


This time, instead of starting with the top crate and moving it, then moving to the next one down (i.e. the new top crate), we want to start by moving the bottom one first. We can do this by creating a new function that returns the index in the pandas df of the nth crate down rather than the top crate. This crate is the one that should be moved first, followed by the n-1 etc.

In [8]:
# New copy of the df
df2 = df.copy()

In [9]:
def get_nth_crate(df, column, n):
    # Keep only the crates that are in the column of interest
    column_crates = df[df['Column'] == column]
        
    # Find the nth heighest crates in that column
    nth_heighest_crates = column_crates['Height'].nlargest(n)
    
    # Return the row index the nth heighest crate 
    return nth_heighest_crates.index[n-1]

Rewrite the instructions to alter how we move the crates. We essentially need to start by finding the nth heighest crate, then n-1th etc, so just make the range count down instead of up to get this order. Can then just use our new function to get the corresponding crates

In [10]:
def do_instruction2(df, instruction):
    for i in range(int(instruction[0]),0,-1):
        # Find the crate that we need to move
        source_index = get_nth_crate(df, int(instruction[1]), i)

        # Move the crate to the target column
        move_crate(df, source_index, int(instruction[2]))

Now we just carry out all the instructions as before and can get the answer we are looking for

In [11]:
for instruction in instructions:
    do_instruction2(df2, instruction)

In [12]:
answer2 = ""
for column in range(1, df2.max()['Column']+1):
    _,_,top_crate_content = get_top_crate(df2, column)
    answer2 += top_crate_content
    
answer2

'WZMFVGGZP'